#### Test mynds packages in the sandbox

In [6]:
from pathlib import Path

import asdf
import numpy as np
import polars as pl
import tqdm

from mynd.backend import metashape
from mynd.collections import GroupID
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def main() -> None:
    """Main entrypoint."""

    PROJECT: Path = Path(
        "/data/kingston_snv_01/acfr_metashape_projects/qdch0ftq_aligned_with_metadata.psz"
    )
    OUTPUT: Path = Path("/data/kingston_snv_01/camera_export")

    assert PROJECT.exists(), f"project does not exist: {PROJECT}"

    match metashape.load_project(PROJECT):
        case Ok(None):
            pass
        case Err(message):
            logger.error(message)

    identifier: GroupID = GroupID(key=0, label="qdch0ftq_20100428_020202")
    cameras: CameraGroup = metashape.camera_services.retrieve_camera_group(
        identifier
    ).unwrap()

    # TODO: Write your code here


# INVOKE MAIN
main()

2024-10-30 09:05:52.638 | ERROR    | __main__:main:246 - backend already has a loaded project
2024-10-30 09:05:56.268 | INFO     | __main__:main:255 - GroupIdentifier(key=0, label='qdch0ftq_20100428_020202')
